<a href="https://colab.research.google.com/github/dhanayanshankar/1on1-questions/blob/master/t5_flan_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install wikipedia
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util


import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


import torch
import sys
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
tokenizer.model_max_length = sys.maxsize
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base").to(device)
sentence_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


def query_LLM(query, context, tok_len):
    t5query = f"""Question: "{query}" Context: {context}"""
    inputs = tokenizer(t5query, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=tok_len)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=03b93cf09e806af18b2b0ab4cf8b0dd68328a0cd7082b5aebaa80c18edc0d895
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8b7b2fa47f2d95489ee3e181323766c6a

# Specify questions here


In [ ]:
import wikipedia
page = wikipedia.page("Albert Einstein")
paras = page.content.split('\n')
filtered_paras = []
for i,p in enumerate(paras):
  if i < 10 or "socialism" in p or "politic" in p:
    filtered_paras.append(p)

context_data = "\n".join(filtered_paras)


In [ ]:
questions = [
    "When was Einstein born?",
    "What was his major achievement in physics?",
    "What was his political view?"
]

results = {question:query_LLM(question,  context_data, 50)  for question in questions}
print("Output:")
print(results)

Output:
{'When was Einstein born?': ['14 March 1879'], 'What was his major achievement in physics?': ['his discovery of the law of the photoelectric effect'], 'What was his political view?': ['in favor of socialism and critical of capitalism']}


In [ ]:
result_answers = []
for key in results.keys():
    result_answers.append(results[key][0])

NameError: ignored

In [ ]:
user_answers = [
    "March 14, 1879",
    "Discovery of photoelectric effect",
    "Socialism. Critical of capitalism"
]


In [ ]:
embeddings1 = sentence_model.encode(user_answers, convert_to_tensor=True)
embeddings2 = sentence_model.encode(result_answers, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)


In [ ]:
for i in range(len(cosine_scores)):
  print(cosine_scores[i][i].item())

0.8786711096763611
0.8453080058097839
0.7965183258056641
